# core

> This is the core module, I haven't decided if I need any other modules yet

List of things we'd like to log:

* Code version
* Uncommitted changes
* Tree directory snapshot
* All shell output logs
* Time and date
* Docker container info
* All code using wandb to grab code
* Any config files mentioned in argv
* nvidia-smi (this is in some experiments anyway)
* location of the local storage with the detailed logs
* hostname

This is mostly a wandb wrapper, with the goal being to copy the wandb directory to some location (such as Onedrive) where it can be stored indefinitely, along with a UUID identifier. At the same time it writes the output log and short job info to a shared git repo. Then we can commit these logs and use the git repo as a continuous log of all experiments run, with the UUIDs available so it would be possible to ask for more info from whoever ran the experiment and they may have it saved.

It might become a huge storage hog but I'm willing to take that risk.

## Structure

Before running anything you run `profane setup` to specify where to save:

* Complete logs (some directory with enough storage space)
* Minimal shared logs (should be a shared git repo)

`profane.core.init` is the entrypoint, it's supposed to act like `wandb.init`.

When initialized:

* It uses the run dir name generated by wandb as the subdir name so it matches
* Creates output directory according to the name in the users subdirectory in the shared repository
* Creates mirrored output directory in the local complete log directory

When the run finishes:

* Extracts the `output.log` and saves it to the shared log directory from before, along with the command run and the metadata
* Copies the entire `wandb` run directory to complete log directory

To do this it has to add a teardown hook that will run on the run finishing.

## To do

* another init safe for distributed (`init_rank_0`) to reduce loc changed
* add unsafe option to save the last hundred lines executed in the shell `history`
* find a way to log stuff that happens in a subprocess, I tested and this won't capture it
* various things could be written at the start of the run instead of at the end, because they don't change
* ~~unpack the `.wandb` protobuf history to csv~~

# Setup Tools

> This code is for setting up where to save the logs, it sets the location of the `shared` and `local` directories in a config file called `.profanerc` in the user's home directory.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

The `get_rcdir` function mainly exists because we need to be able to set a global variable for
the directory to save the config file to when we run tests. Most of the time, it will just return
the home directory.

In [ ]:
#| export
import argparse
from pathlib import Path

# this is used for testing
RCDIR = None

def get_rcdir():
    """Function to get the rcdir. Returns `Path` object of the rcdir."""
    rcdir = RCDIR or Path.home()
    if isinstance(rcdir, str):
        rcdir = Path(rcdir)
    return rcdir

Argument parser for the `profane_setup` shell command.

In [ ]:
#| exports
def parse_args():
    argument_callback = argparse.ArgumentParser("Set up a ~/.profanerc file and register the required directories.")
    argument_callback.add_argument('local_storage', type=Path, help="The local storage directory, everything will be stored here.")
    argument_callback.add_argument('shared_storage', type=Path, help="The shared storage directory, only the terminal logs and metadata will be stored here.")
    argument_callback.add_argument('--user', type=str, default=None, help="Optional: The user name to use for the shared storage directory.")
    return argument_callback.parse_args()

In [ ]:
#| export
def setup():
    """Parses args and calls write_profanerc()"""
    args = parse_args() 
    return write_profanerc(args.local_storage, args.shared_storage, args.user)

In [ ]:
#| export
def write_profanerc(local_storage, shared_storage, user):
    """
    Function to set up the ~/.profanerc file and register the required directories.
    Example ~/.profanerc file that will be written:
    ```
        local_storage=/home/user/profane/local_storage
        shared_storage=/home/user/profane/shared_storage
        user=me
    ```
    """
    # check the two directories exist
    if not local_storage.exists():
        raise FileNotFoundError(f"{local_storage} does not exist.")
    if not shared_storage.exists():
        raise FileNotFoundError(f"{shared_storage} does not exist.")
    # save the config file
    rcdir = get_rcdir()
    config_file = rcdir / ".profanerc"
    config_file.write_text(f"local_storage={str(local_storage.resolve())}\nshared_storage={str(shared_storage.resolve())}" + (f"\nuser={user}" if user else ""))

In [ ]:
#| export
def get_config():
    """
    Function to load the config from the ~/.profanerc file.
    Loads the config as a dictionary and returns it.
    Example ~/.profanerc file to load from:
    ```
        local_storage=/home/user/profane/local_storage
        shared_storage=/home/user/profane/shared_storage
        user=me
    ```
    Example config dictionary:
    ```
       {'local_storage': '/home/user/profane/local_storage',
        'shared_storage': '/home/user/profane/shared_storage',
        'user': 'me'}
    ```
    """
    rcdir = get_rcdir()
    config_file = rcdir / ".profanerc"
    if not config_file.exists():
        raise FileNotFoundError(f"{config_file} does not exist. Run profanewrite_profanerc <local dir> <shared dir> to set it up.")
    config = {}
    for line in config_file.read_text().splitlines():
        key, value = line.split("=")
        config[key] = value
    return config

# Opening `.wandb` files

> To log some useful values we need to be able to open `.wandb` files and read them.

`wandb` saves its data in a protobuf format, which is a binary format. This module contains a
function to open the protobuf file and return a dictionary of the values.

In [ ]:
#| export
from wandb.proto import wandb_internal_pb2
from wandb.sdk.internal import datastore

In [ ]:
#| export
def iterate_records(data_path):
    """Iterates over wandb's protobuf records in `.wandb` files."""
    # https://github.com/wandb/wandb/issues/1768#issuecomment-976786476 
    ds = datastore.DataStore()
    ds.open_for_scan(data_path)
    terminal_log = []

    data = ds.scan_record()
    while data is not None:
        pb = wandb_internal_pb2.Record()
        try:
            pb.ParseFromString(data[1])  
            yield pb
        except wandb_internal_pb2.google.protobuf.message.DecodeError:
            data = ds.scan_record()
            continue
        data = ds.scan_record()

To test this function I'm going to run `wandb` offline in a temporary directory, then iterate over the file it creates:

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path
import wandb

def inspect_wandb_file(f):
    # run function f on wandb file that is generated
    with TemporaryDirectory() as tmpdirname:
        tmpdirname = Path(tmpdirname)
        print("created ", tmpdirname)
       # wandb by default saves to a directory in the current working directory so
        # we need to pass a path to the directory we just created
        target_dir = tmpdirname / 'wandb'
        print(target_dir)
        run = wandb.init(dir=tmpdirname, mode="offline")
        for i in range(10):
            wandb.log({'test': i})
        run.finish()
        # find the wandb file
        wandb_file_path = None
        for fpath in target_dir.rglob('*'):
            if fpath.suffix == '.wandb':
                wandb_file_path = fpath
        # call f
        return f(wandb_file_path)

def history(wandb_file_path):
    for pb in iterate_records(wandb_file_path):
        record_type = pb.WhichOneof('record_type')
        if record_type == "history":
            print(pb)

inspect_wandb_file(history)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


created  /var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmpse8hgc5d
/var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmpse8hgc5d/wandb


test,▁▂▃▃▄▅▆▆▇█
test,9


num: 5
history {
  item {
    key: "test"
    value_json: "0"
  }
  item {
    key: "_timestamp"
    value_json: "1694554865.618004"
  }
  item {
    key: "_runtime"
    value_json: "0.7110311985015869"
  }
  item {
    key: "_step"
    value_json: "0"
  }
  step {
  }
}

num: 6
history {
  item {
    key: "test"
    value_json: "1"
  }
  item {
    key: "_timestamp"
    value_json: "1694554865.618102"
  }
  item {
    key: "_runtime"
    value_json: "0.7111291885375977"
  }
  item {
    key: "_step"
    value_json: "1"
  }
  step {
    num: 1
  }
}

num: 7
history {
  item {
    key: "test"
    value_json: "2"
  }
  item {
    key: "_timestamp"
    value_json: "1694554865.618141"
  }
  item {
    key: "_runtime"
    value_json: "0.7111680507659912"
  }
  item {
    key: "_step"
    value_json: "2"
  }
  step {
    num: 2
  }
}

num: 8
history {
  item {
    key: "test"
    value_json: "3"
  }
  item {
    key: "_timestamp"
    value_json: "1694554865.618173"
  }
  item {
    key: "_ru

From this it looks like all I need to do is turn each `history` entry into a row in a csv.
Should be easy by just making each row into a dictionary and then deciding on an ordering for the columns.

In [ ]:
#| export
import json
from collections import defaultdict

def history_csv(wandb_file_path):
    rows = []
    for pb in iterate_records(wandb_file_path):
        record_type = pb.WhichOneof('record_type')
        if record_type == "history":
            # iterate over `item` entries in this record
            row = defaultdict(lambda: float('nan')) # default value is nan
            for item in pb.history.item:
                row[item.key] = json.loads(item.value_json)
            rows.append(row)
    if len(rows) == 0:
        return ""
    cols = sorted(list(set([k for r in rows for k in r.keys()]))) # get all unique keys
    # put "_step" first
    cols.remove("_step")
    cols.insert(0, "_step")
    csv_str = ""
    csv_str += ",".join(cols) + "\n"
    for row in rows:
        csv_str += ",".join(json.dumps(row[k]) for k in cols) + "\n"
    return csv_str

Testing this function, it should print the contents of the history as a csv:

In [ ]:
print(inspect_wandb_file(history_csv))

created  /var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmpwxx63z9b
/var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmpwxx63z9b/wandb


test,▁▂▃▃▄▅▆▆▇█
test,9


_step,_runtime,_timestamp,test
0,1.365873098373413,1694554868.371184,0
1,1.36592698097229,1694554868.371238,1
2,1.3659608364105225,1694554868.3712718,2
3,1.3659908771514893,1694554868.371302,3
4,1.366011142730713,1694554868.3713222,4
5,1.3660311698913574,1694554868.3713422,5
6,1.3660590648651123,1694554868.37137,6
7,1.3660800457000732,1694554868.371391,7
8,1.366105079650879,1694554868.371416,8
9,1.3661339282989502,1694554868.371445,9



We also want to extract the output log containing all the shell output, then write that to a text file:

In [ ]:
#| export
def parse_output_log(data_path):
    """
    Parse wandb data from a given path.
    Returns the terminal log typically saved as `output.log`,
    which isn't created unless you're running in online mode.
    But, the data still exists in the `.wandb` file.
    """
    terminal_log = []
    for pb in iterate_records(data_path):
        record_type = pb.WhichOneof("record_type")
        if record_type == "output_raw":
            terminal_log.append(pb.output_raw.line)
    return "".join(terminal_log)

And we want to extract the system traces, like GPU utilization, and write that to a csv:

In [ ]:
#| export
from collections import OrderedDict

def parse_stats(data_path):
    """
    Parse `.wandb` file to extract the system stats,
    such as CPU, GPU, memory, etc.
    """
    rows = OrderedDict()
    names = []
    min_seconds = None
    for pb in iterate_records(data_path):
        if pb.stats:
            row = {}
            n = 0
            for stat in pb.stats.item:
                if stat.key not in names:
                    names.append(stat.key)
                row[stat.key] = stat.value_json
                n += 1
            if n > 0:
                if min_seconds is None:
                    min_seconds = pb.stats.timestamp.seconds
                seconds = pb.stats.timestamp.seconds - min_seconds
                rows[seconds] = row 
    # sort each row using names
    _rows = []
    for k, v in rows.items():
        _rows.append([k]+[v.get(name, None) for name in names])
    rows = _rows
    # build csv
    header = ['relative_seconds'] + names
    return ', '.join(header) + '\n' + '\n'.join([', '.join([str(v) for v in row]) for row in rows])

# Substitute `wandb.init`

> This is a wrapper for `wandb.init`, it mostly sets up callbacks that get triggered on exit.

In [ ]:
#| export
import wandb
import shutil
import os
import atexit
import logging
import subprocess
import sys
import json

from distutils.dir_util import copy_tree
from wandb.sdk.wandb_run import TeardownHook, TeardownStage

# don't print anything by default
logging.basicConfig(format='%(asctime)s %(message)s %(name)s', level=logging.WARNING)
logger = logging.getLogger(__name__)

In [ ]:
#| export
def create_dirs(run_id, user, project):
    """Create shared and local sub-directories for a run
    in the shared and local storage directories.
    Returns the Paths to the shared and local directories."""
    config = get_config()
    if len(project) == 0:
        project = "misc"
    shared_dir = Path(config['shared_storage']) / user / project / run_id
    shared_dir.mkdir(parents=True)
    local_dir = Path(config['local_storage']) / project / run_id
    local_dir.mkdir(parents=True)
    return shared_dir, local_dir

In [ ]:
#| export
class SyncLocalCallback:
    """Callback to sync wandb dir to local storage."""
    def __init__(self):
        self.exited = False

    def register_dirs(self, local_dir, wandb_dir):
        self.local_dir = local_dir
        self.wandb_dir = wandb_dir

    def __call__(self):
        logger.info("local callback called")
        if not self.exited:
            # if dir is empty sleep
            # not clear if this is necessary
            # while len([p for p in self.wandb_dir.iterdir()]) < 1:
            #     time.sleep(1)
            #     print(f"waiting for {self.wandb_dir} to be populated")
            logger.info(f"found {len([p for p in self.wandb_dir.iterdir()])} files in {self.wandb_dir}")
            # copy wandb dir to shared dir, nothing fancy
            # copy_tree(str(self.wandb_dir), str(self.local_dir), verbose=1)
            for path_object in self.wandb_dir.rglob('*'):
                if path_object.is_file():
                    logger.info(f"copying {path_object} to {self.local_dir / path_object.relative_to(self.wandb_dir)}")
                    shutil.copy(path_object, self.local_dir / path_object.relative_to(self.wandb_dir))
                else:
                    (self.local_dir / path_object.relative_to(self.wandb_dir)).mkdir(parents=True, exist_ok=True)
            logger.info(f"copied {self.wandb_dir} to {self.local_dir}")
            # find the wandb file
            wandb_file_path = None
            for fpath in self.local_dir.rglob('*'):
                if fpath.suffix == '.wandb':
                    wandb_file_path = fpath
            # extract the history as a csv
            history_csv_str = history_csv(wandb_file_path)
            # write the csv to the same location as the wandb
            # file but with a .csv extension
            csv_path = wandb_file_path.with_suffix('.csv')
            csv_path.write_text(history_csv_str)
            logger.info(f"wrote {csv_path}")
            self.exited = True

In [ ]:
#| export
class SyncSharedCallback:
    """Callback to sync the shared directory with the wandb directory, contains `callback`s that it calls when called."""
    def __init__(self):
        self.exited = False
        self.callbacks = []

    def register_dirs(self, shared_dir, wandb_dir):
        self.shared_dir = shared_dir
        self.wandb_dir = wandb_dir

    def register_callback(self, callback):
        """
        Register functions to act as callbacks on `.wandb` files.
        That's right, my callback has callbacks.
        Expect these to return a string to save and a filename to save it as.
        """
        self.callbacks.append(callback)

    def __call__(self):
        logger.info("shared callback called")
        if not self.exited:
            # find the wandb output file:
            for path_object in self.wandb_dir.rglob('*'):
                if path_object.is_file():
                    if path_object.suffix == '.wandb':
                        wandb_file = path_object
                        break
            # parse the wandb file
            state = {'wandb_file': wandb_file}
            for callback in self.callbacks:
                s, filename = callback(state)
                # write to shared_dir
                with open(self.shared_dir / filename, 'w') as f:
                    f.write(s)
                logger.info(f"{self.shared_dir / filename} written")
            # iterate over files in wandb/files directory
            for path_object in (self.wandb_dir / 'files').rglob('*'):
                if path_object.is_file():
                    # copy file to shared_dir
                    shutil.copy(path_object, self.shared_dir)
                    logger.info(f"{path_object} copied to {self.shared_dir}")
            self.exited = True

In [ ]:
#| export
def system_to_string(command):
    """Run system command, capture and decode the output to string."""
    if type(command) == str:
        command = command.split(' ')
    return subprocess.check_output(command).decode('utf-8')

In [ ]:
#| export
def git_patch_callback():
    """Callback that creates a patch for the changes in the current directory."""
    return system_to_string(['git', 'diff', 'HEAD'])

In [ ]:
#| export
def git_status():
    "Git status in currently directory"
    return system_to_string(['git', 'status'])

def git_branch():
    "Current branch"
    return system_to_string(["git", "rev-parse", "--abbrev-ref", "HEAD"])

In [ ]:
#| export
def init(**kwargs):
    """
    A wrapper for `wandb.init`. kwargs are passed to `wandb.init` with the
    following modifications:

    - if not specified `mode` is set to `offline`
    - kwargs prefixed by "profane_" are stripped of the prefix and used to
        configure the `profane` hooks:
        - `profane_save`: a list of paths to plaintext files to save to 
        the shared directory, for example `profane_save=['README.md']`
    """
    # separate profane kwargs from wandb kwargs
    pkwargs = {}
    for k, v in kwargs.items():
        if k.startswith('profane_'):
            pkwargs[k[8:]] = v
    kwargs = {k: v for k, v in kwargs.items() if not k.startswith('profane_')}
    # initialize hooks
    local_hook = SyncLocalCallback()
    shared_hook = SyncSharedCallback()
    # initialize wandb
    if 'mode' not in kwargs: 
        kwargs['mode'] = 'offline'
    run = wandb.init(**kwargs)
    # configure hooks
    config = get_config()
    wandb_dir = Path(run.dir).parent
    username = os.environ['USER'] if 'user' not in config else config['user']
    shared_dir, local_dir = create_dirs(wandb_dir.name, username, run.project)
    shared_hook.register_dirs(shared_dir, wandb_dir)
    shared_hook.register_callback(lambda s: (parse_output_log(s['wandb_file']), 'output.log'))
    shared_hook.register_callback(lambda s: (parse_stats(s['wandb_file']), 'system_stats.csv'))
    shared_hook.register_callback(lambda s: (git_patch_callback(), 'git.patch'))
    shared_hook.register_callback(lambda s: (git_status(), 'git.status'))
    # extract config if it exists
    if 'config' in kwargs:
        json_config = json.dumps(kwargs['config'], indent=4, sort_keys=True)
        shared_hook.register_callback(lambda s: (json_config, 'config.json'))
    # register save paths
    if 'save' in pkwargs:
        for path in pkwargs['save']:
            if isinstance(path, str):
                path = Path(path)
            assert path.exists(), f"Path {path} does not exist"
            assert path.is_file(), f"Path {path} is not a file"
            # assuming path is plaintext
            shared_hook.register_callback(lambda s, path=path: (path.read_text(), path.name))
    shared_hook.register_callback(lambda s: (' '.join(sys.argv), 'argv.txt'))
    local_hook.register_dirs(local_dir, wandb_dir)
    # this will trigger if run.finish() is called
    def report_exit():
        print(f"profane: logs saved to {shared_dir}")
        print(f"profane: wandb copied to {local_dir}")
    run._teardown_hooks += [TeardownHook(local_hook, TeardownStage.LATE),
                            TeardownHook(shared_hook, TeardownStage.LATE),
                            TeardownHook(report_exit, TeardownStage.LATE)]
    def finish():
        logger.info("atexit profane.init finish hook called")
        run.finish()
    # this is necessary because wandb doesn't use the teardown hooks otherwise
    # so it just kind of forces it to happen
    atexit.register(finish)
    return run

# Tests

In [ ]:
# to test this I need to make a temporary directory and run a fake experiment
# then I can check that whatever gets printed is exactly what is stored in the
# wandb file
from tempfile import TemporaryDirectory
from pathlib import Path
import os
os.environ['WANDB_SILENT'] = 'true'

# import subprocess

printed = []
def _print(*args, **kwargs):
    global printed
    printed += [*args, "\n"]
    return print(*args, **kwargs)

def test_experiment():
    global printed
    global RCDIR
    with TemporaryDirectory() as tmpdirname:
        print("created ", tmpdirname)
        RCDIR = tmpdirname
        tmpdirname = Path(tmpdirname)
        local_storage = Path(tmpdirname/'profane_storage')
        local_storage.mkdir()
        shared_storage = Path(tmpdirname/'profane_shared_storage')
        shared_storage.mkdir()
        write_profanerc(local_storage, shared_storage, None)
        with open(tmpdirname / '.profanerc', 'r') as f:
            print(f.read())
        print('created temporary directory', tmpdirname)
        # wandb by default saves to a directory in the current working directory so
        # we need to pass a path to the directory we just created
        target_dir = tmpdirname / 'wandb'
        print(target_dir)
        run = init(dir=tmpdirname)
        wandb.log({'test': 1})
        _print("something to be logged")
        for i in range(10):
            _print(f"logging {i}")
        run.finish()
        # check that the files that should have been created are there
        wandb_file_exists = False
        for fpath in local_storage.rglob('*'):
            if fpath.suffix == '.wandb':
                wandb_file_exists = True
        assert wandb_file_exists, f"{local_storage} does not contain a .wandb file"
        output_log_exists = False
        for fpath in shared_storage.rglob('*'):
            if fpath.name == 'output.log':
                output_log_exists = True
                with open(fpath, 'r') as f:
                    output_log = f.read()
        assert output_log_exists
        # check that the output log is correct
        assert output_log == ''.join(printed), f"output log does not match printed output\n{output_log}\n{''.join(printed)}"

test_experiment()

created  /var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmpgpq31bf7
local_storage=/private/var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmpgpq31bf7/profane_storage
shared_storage=/private/var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmpgpq31bf7/profane_shared_storage
created temporary directory /var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmpgpq31bf7
/var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmpgpq31bf7/wandb


something to be logged
logging 0
logging 1
logging 2
logging 3
logging 4
logging 5
logging 6
logging 7
logging 8
logging 9


test,▁
test,1


profane: logs saved to /private/var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmpgpq31bf7/profane_shared_storage/gngdb/misc/offline-run-20230912_174109-ka5w6cay
profane: wandb copied to /private/var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmpgpq31bf7/profane_storage/misc/offline-run-20230912_174109-ka5w6cay


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

**WARNING**: this test may run different code if you didn't save and run the above cell beforehand

In [ ]:
# the same test as above but run on the command line
os.environ['WANDB_SILENT'] = 'false'

with TemporaryDirectory() as tmpdirname:
    RCDIR = tmpdirname
    tmpdirname = Path(tmpdirname)
    local_storage = Path(tmpdirname/'profane_storage')
    local_storage.mkdir()
    shared_storage = Path(tmpdirname/'profane_shared_storage')
    shared_storage.mkdir()
    write_profanerc(local_storage, shared_storage, None)

    script = f"""
import profane.core
profane.core.RCDIR = '{tmpdirname}'
profane.core.init(dir='{tmpdirname}')
print('done')
"""
    tmpdir = Path(tmpdirname)
    with open(tmpdir/'test.py', 'w') as f:
        f.write(script)
    subprocess.run(['python', tmpdir/'test.py'])
    #for fpath in local_storage.rglob('*'):
    #    print(fpath)
    # for fpath in shared_storage.rglob('*'):
    #     print(fpath)
    # for fpath in tmpdir.rglob('*'):
    #     print(fpath)
    wandb_file_exists = False
    for fpath in local_storage.rglob('*'):
        if fpath.suffix == '.wandb':
            wandb_file_exists = True
    assert wandb_file_exists, f"{local_storage} does not contain a .wandb file"
    output_log_exists = False
    for fpath in shared_storage.rglob('*'):
        if fpath.name == 'output.log':
            output_log_exists = True
            with open(fpath, 'r') as f:
                output_log = f.read()
    assert output_log_exists
    # check that the output log is correct
    assert output_log == "done\n", f"output log is incorrect: {output_log}"

wandb: Tracking run with wandb version 0.15.7
wandb: W&B syncing is set to `offline` in this directory.  
wandb: Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.
wandb: Waiting for W&B process to finish... (success).


done


wandb: You can sync this run to the cloud by running:
wandb: wandb sync /var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmpg5zvdy7o/wandb/offline-run-20230912_174112-yg6z9o3p
wandb: Find logs at: /var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmpg5zvdy7o/wandb/offline-run-20230912_174112-yg6z9o3p/logs


profane: logs saved to /private/var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmpg5zvdy7o/profane_shared_storage/gngdb/misc/offline-run-20230912_174112-yg6z9o3p
profane: wandb copied to /private/var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmpg5zvdy7o/profane_storage/misc/offline-run-20230912_174112-yg6z9o3p
/var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmpg5zvdy7o/profane_shared_storage/gngdb
/var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmpg5zvdy7o/profane_shared_storage/gngdb/misc
/var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmpg5zvdy7o/profane_shared_storage/gngdb/misc/offline-run-20230912_174112-yg6z9o3p
/var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmpg5zvdy7o/profane_shared_storage/gngdb/misc/offline-run-20230912_174112-yg6z9o3p/requirements.txt
/var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmpg5zvdy7o/profane_shared_storage/gngdb/misc/offline-run-20230912_174112-yg6z9o3p/git.status
/var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmpg5zvdy7o/profane_shared_